In [8]:
import logging
from typing import Any, List, Sequence, Tuple

import tqdm
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras
from tensorflow.keras import layers

from data.data_prep import prepare_data
from basic_model import create_dataset_x, load_model
from agent_env import StockTradingEnv

In [9]:
sector = 'tech'
model_name = 'tech_lstm'
look_back = 50
seed = 10

gamma = 0.99  # Discount factor for past rewards
eps = np.finfo(np.float32).eps.item()  # Smallest number such that 1.0 + eps != 1.0

In [10]:
def get_dataset(sector, model_name, look_back):
    dates, close_prices, normalised_close_prices, N_tickers, normaliser, symbols = prepare_data(sector, split=False,save_model_stocks=False)

    x = create_dataset_x(normalised_close_prices, look_back)
    y_test_real = close_prices[look_back:]

    model = load_model(model_name)
    y_pred = model.predict(x)

    d = {symbol: close_price for symbol, close_price in zip(symbols, y_test_real.T)}
    d.update({f"{symbol}_grad": pred_grad for symbol, pred_grad in zip(symbols, y_pred.T)})

    df = pd.DataFrame(data=d)

    return df

data_df = get_dataset(sector, model_name, look_back)

In [11]:
env = StockTradingEnv(data_df)
env.seed(seed)

In [16]:
class Critic(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.d1 = tf.keras.layers.Dense(64, activation='sigmoid')
        self.d2 = tf.keras.layers.Dense(64, activation='sigmoid')
        self.v = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, input_data):
        x = self.d1(input_data)
        x = self.d2(x)
        v = self.v(x)
        return v
    

class Actor(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.d1 = tf.keras.layers.Dense(64, activation='sigmoid')
        self.d2 = tf.keras.layers.Dense(64, activation='sigmoid')
        self.a = tf.keras.layers.Dense(env.action_space.n, activation='softmax')

    def call(self, input_data):
        x = self.d1(input_data)
        x = self.d2(x)
        a = self.a(x)
        return a

class Agent():
    def __init__(self, gamma = 0.99):
        self.gamma = gamma
        self.a_opt = tf.keras.optimizers.Adam(learning_rate=5e-6)
        self.c_opt = tf.keras.optimizers.Adam(learning_rate=5e-5)
        self.actor = Actor()
        self.critic = Critic()

    def act(self, state):
        action = self.actor(np.array([state]))[0]
        return action

    def actor_loss(self, prob, action, td):
        dist = tfp.distributions.Categorical(probs=prob, dtype=tf.float32)
        log_prob = dist.log_prob(action)
        loss = -log_prob * td

        return loss

    def learn(self, state, action, reward, next_state, done):
        state = np.array([state])
        next_state = np.array([next_state])
        with tf.GradientTape() as tape1, tf.GradientTape() as tape2:
            p = self.actor(state, training=True)
            v = self.critic(state, training=True)
            vn = self.critic(next_state, training=True)
            td = reward + self.gamma * vn * (1 - int(done)) - v
            a_loss = self.actor_loss(p, action, td)
            c_loss = td**2
            grads1 = tape1.gradient(a_loss, self.actor.trainable_variables)
            grads2 = tape2.gradient(c_loss, self.critic.trainable_variables)
        self.a_opt.apply_gradients(zip(grads1, self.actor.trainable_variables))
        self.c_opt.apply_gradients(zip(grads2, self.critic.trainable_variables))

        return a_loss, c_loss

    def save_agent(self):
        self.actor.save(filepath='models/actor')
        self.critic.save(filepath='models/critic')
        

In [17]:
agent = Agent()
steps = 1000
best_reward = 0
for s in range(steps):
    done = False
    state = env.reset()
    all_aloss = []
    all_closs = []

    for idx in range(100):
        #env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        aloss, closs = agent.learn(state, action, reward, next_state, done)
        closs_print = round(float(closs.numpy()[0][0]), 2)
        aloss_print = round(sum(sum(aloss.numpy())), 2)
        print(f"Run: {s}, Day: {idx}, closs: {closs_print}, aloss: {aloss_print}, return: {round(float(reward), 2)}")
        all_aloss.append(aloss)
        all_closs.append(closs)
        state = next_state

    if reward > best_reward:
        best_reward = reward
        agent.save_agent()
        if done:
            #print("total step for this episord are {}".format(t))
            print("total reward after {} steps is {}".format(s, total_reward))

Run: 0, Day: 0, closs: 9999.28, aloss: 21970.51, return: 100.0
Run: 0, Day: 1, closs: 9999.28, aloss: 21967.02, return: 100.0
Run: 0, Day: 2, closs: 9999.28, aloss: 21963.53, return: 100.0
Run: 0, Day: 3, closs: 9999.28, aloss: 21960.05, return: 100.0
Run: 0, Day: 4, closs: 9999.28, aloss: 21956.57, return: 100.0
Run: 0, Day: 5, closs: 9999.28, aloss: 21953.08, return: 100.0
Run: 0, Day: 6, closs: 9999.28, aloss: 21949.59, return: 100.0
Run: 0, Day: 7, closs: 9999.28, aloss: 21946.1, return: 100.0
Run: 0, Day: 8, closs: 9999.28, aloss: 21942.62, return: 100.0
Run: 0, Day: 9, closs: 9999.28, aloss: 21939.13, return: 100.0
Run: 0, Day: 10, closs: 9999.28, aloss: 21935.64, return: 100.0
Run: 0, Day: 11, closs: 9999.28, aloss: 21932.14, return: 100.0
Run: 0, Day: 12, closs: 9999.28, aloss: 21928.65, return: 100.0
Run: 0, Day: 13, closs: 9999.28, aloss: 21925.17, return: 100.0
Run: 0, Day: 14, closs: 9999.28, aloss: 21921.69, return: 100.0
Run: 0, Day: 15, closs: 9999.28, aloss: 21918.2, re

Run: 1, Day: 27, closs: 9999.25, aloss: 21527.98, return: 100.0
Run: 1, Day: 28, closs: 9999.25, aloss: 21524.5, return: 100.0
Run: 1, Day: 29, closs: 9999.25, aloss: 21521.02, return: 100.0
Run: 1, Day: 30, closs: 9999.25, aloss: 21517.53, return: 100.0
Run: 1, Day: 31, closs: 9999.25, aloss: 21514.05, return: 100.0
Run: 1, Day: 32, closs: 9999.25, aloss: 21510.57, return: 100.0
Run: 1, Day: 33, closs: 9999.25, aloss: 21507.09, return: 100.0
Run: 1, Day: 34, closs: 9999.25, aloss: 21503.6, return: 100.0
Run: 1, Day: 35, closs: 9999.25, aloss: 21500.1, return: 100.0
Run: 1, Day: 36, closs: 9999.25, aloss: 21496.62, return: 100.0
Run: 1, Day: 37, closs: 9999.25, aloss: 21493.14, return: 100.0
Run: 1, Day: 38, closs: 9999.25, aloss: 21489.65, return: 100.0
Run: 1, Day: 39, closs: 9999.25, aloss: 21486.18, return: 100.0
Run: 1, Day: 40, closs: 9999.25, aloss: 21482.71, return: 100.0
Run: 1, Day: 41, closs: 9999.25, aloss: 21479.26, return: 100.0
Run: 1, Day: 42, closs: 9999.25, aloss: 214

Run: 2, Day: 55, closs: 9999.22, aloss: 21083.21, return: 100.0
Run: 2, Day: 56, closs: 9999.22, aloss: 21079.82, return: 100.0
Run: 2, Day: 57, closs: 9999.22, aloss: 21076.4, return: 100.0
Run: 2, Day: 58, closs: 9999.21, aloss: 21072.95, return: 100.0
Run: 2, Day: 59, closs: 9999.21, aloss: 21069.5, return: 100.0
Run: 2, Day: 60, closs: 9999.21, aloss: 21066.04, return: 100.0
Run: 2, Day: 61, closs: 9999.21, aloss: 21062.58, return: 100.0
Run: 2, Day: 62, closs: 9999.21, aloss: 21059.11, return: 100.0
Run: 2, Day: 63, closs: 9999.21, aloss: 21055.65, return: 100.0
Run: 2, Day: 64, closs: 9999.21, aloss: 21052.18, return: 100.0
Run: 2, Day: 65, closs: 9999.21, aloss: 21048.71, return: 100.0
Run: 2, Day: 66, closs: 9999.21, aloss: 21045.24, return: 100.0
Run: 2, Day: 67, closs: 9999.21, aloss: 21041.77, return: 100.0
Run: 2, Day: 68, closs: 9999.21, aloss: 21038.31, return: 100.0
Run: 2, Day: 69, closs: 9999.21, aloss: 21034.85, return: 100.0
Run: 2, Day: 70, closs: 9999.21, aloss: 21

Run: 3, Day: 84, closs: 9999.18, aloss: 20636.13, return: 100.0
Run: 3, Day: 85, closs: 9999.18, aloss: 20632.66, return: 100.0
Run: 3, Day: 86, closs: 9999.18, aloss: 20629.22, return: 100.0
Run: 3, Day: 87, closs: 9999.18, aloss: 20625.78, return: 100.0
Run: 3, Day: 88, closs: 9999.18, aloss: 20622.35, return: 100.0
Run: 3, Day: 89, closs: 9999.18, aloss: 20618.91, return: 100.0
Run: 3, Day: 90, closs: 9999.18, aloss: 20615.47, return: 100.0
Run: 3, Day: 91, closs: 9999.18, aloss: 20612.03, return: 100.0
Run: 3, Day: 92, closs: 9999.18, aloss: 20608.58, return: 100.0
Run: 3, Day: 93, closs: 9999.18, aloss: 20605.14, return: 100.0
Run: 3, Day: 94, closs: 9999.18, aloss: 20601.7, return: 100.0
Run: 3, Day: 95, closs: 9999.18, aloss: 20598.26, return: 100.0
Run: 3, Day: 96, closs: 9999.18, aloss: 20594.84, return: 100.0
Run: 3, Day: 97, closs: 9999.18, aloss: 20591.4, return: 100.0
Run: 3, Day: 98, closs: 9999.18, aloss: 20587.95, return: 100.0
Run: 3, Day: 99, closs: 9999.18, aloss: 20

Run: 5, Day: 14, closs: 9999.14, aloss: 20186.79, return: 100.0
Run: 5, Day: 15, closs: 9999.14, aloss: 20183.37, return: 100.0
Run: 5, Day: 16, closs: 9999.14, aloss: 20179.93, return: 100.0
Run: 5, Day: 17, closs: 9999.14, aloss: 20176.5, return: 100.0
Run: 5, Day: 18, closs: 9999.14, aloss: 20173.07, return: 100.0
Run: 5, Day: 19, closs: 9999.14, aloss: 20169.61, return: 100.0
Run: 5, Day: 20, closs: 9999.14, aloss: 20166.14, return: 100.0
Run: 5, Day: 21, closs: 9999.14, aloss: 20162.65, return: 100.0
Run: 5, Day: 22, closs: 9999.14, aloss: 20159.11, return: 100.0
Run: 5, Day: 23, closs: 9999.14, aloss: 20155.62, return: 100.0
Run: 5, Day: 24, closs: 9999.14, aloss: 20152.17, return: 100.0
Run: 5, Day: 25, closs: 9999.14, aloss: 20148.75, return: 100.0
Run: 5, Day: 26, closs: 9999.14, aloss: 20145.37, return: 100.0
Run: 5, Day: 27, closs: 9999.14, aloss: 20141.98, return: 100.0
Run: 5, Day: 28, closs: 9999.14, aloss: 20138.6, return: 100.0
Run: 5, Day: 29, closs: 9999.14, aloss: 20

...

Run: 350, Day: 12, closs: 9998.0, aloss: 0.18, return: 100.0
Run: 350, Day: 13, closs: 9998.0, aloss: 0.18, return: 100.0
Run: 350, Day: 14, closs: 9998.0, aloss: 0.18, return: 100.0
Run: 350, Day: 15, closs: 9998.0, aloss: 0.18, return: 100.0
Run: 350, Day: 16, closs: 9998.0, aloss: 0.18, return: 100.0
Run: 350, Day: 17, closs: 9998.0, aloss: 0.18, return: 100.0
Run: 350, Day: 18, closs: 9998.0, aloss: 0.18, return: 100.0
Run: 350, Day: 19, closs: 9998.01, aloss: 0.18, return: 100.0
Run: 350, Day: 20, closs: 9998.01, aloss: 0.18, return: 100.0
Run: 350, Day: 21, closs: 9998.01, aloss: 0.18, return: 100.0
Run: 350, Day: 22, closs: 9998.01, aloss: 0.18, return: 100.0
Run: 350, Day: 23, closs: 9998.01, aloss: 0.18, return: 100.0
Run: 350, Day: 24, closs: 9998.01, aloss: 0.18, return: 100.0
Run: 350, Day: 25, closs: 9998.01, aloss: 0.18, return: 100.0
Run: 350, Day: 26, closs: 9998.01, aloss: 0.18, return: 100.0
Run: 350, Day: 27, closs: 9998.01, aloss: 0.18, return: 100.0
Run: 350, Day: 

Run: 351, Day: 46, closs: 9998.11, aloss: 0.17, return: 100.0
Run: 351, Day: 47, closs: 9998.11, aloss: 0.17, return: 100.0
Run: 351, Day: 48, closs: 9998.11, aloss: 0.17, return: 100.0
Run: 351, Day: 49, closs: 9998.11, aloss: 0.17, return: 100.0
Run: 351, Day: 50, closs: 9998.1, aloss: 0.17, return: 100.0
Run: 351, Day: 51, closs: 9998.11, aloss: 0.17, return: 100.0
Run: 351, Day: 52, closs: 9998.1, aloss: 0.17, return: 100.0
Run: 351, Day: 53, closs: 9998.1, aloss: 0.17, return: 100.0
Run: 351, Day: 54, closs: 9998.1, aloss: 0.17, return: 100.0
Run: 351, Day: 55, closs: 9998.1, aloss: 0.17, return: 100.0
Run: 351, Day: 56, closs: 9998.11, aloss: 0.17, return: 100.0
Run: 351, Day: 57, closs: 9998.1, aloss: 0.17, return: 100.0
Run: 351, Day: 58, closs: 9998.11, aloss: 0.17, return: 100.0
Run: 351, Day: 59, closs: 9998.11, aloss: 0.17, return: 100.0
Run: 351, Day: 60, closs: 9998.11, aloss: 0.17, return: 100.0
Run: 351, Day: 61, closs: 9998.11, aloss: 0.17, return: 100.0
Run: 351, Day:

Run: 352, Day: 80, closs: 9998.08, aloss: 0.17, return: 100.0
Run: 352, Day: 81, closs: 9998.08, aloss: 0.17, return: 100.0
Run: 352, Day: 82, closs: 9998.1, aloss: 0.17, return: 100.0
Run: 352, Day: 83, closs: 9998.08, aloss: 0.17, return: 100.0
Run: 352, Day: 84, closs: 9998.07, aloss: 0.17, return: 100.0
Run: 352, Day: 85, closs: 9998.05, aloss: 0.17, return: 100.0
Run: 352, Day: 86, closs: 9998.07, aloss: 0.17, return: 100.0
Run: 352, Day: 87, closs: 9998.07, aloss: 0.17, return: 100.0
Run: 352, Day: 88, closs: 9998.07, aloss: 0.17, return: 100.0
Run: 352, Day: 89, closs: 9998.06, aloss: 0.17, return: 100.0
Run: 352, Day: 90, closs: 9998.07, aloss: 0.17, return: 100.0
Run: 352, Day: 91, closs: 9998.11, aloss: 0.17, return: 100.0
Run: 352, Day: 92, closs: 9998.11, aloss: 0.17, return: 100.0
Run: 352, Day: 93, closs: 9998.12, aloss: 0.17, return: 100.0
Run: 352, Day: 94, closs: 9998.12, aloss: 0.17, return: 100.0
Run: 352, Day: 95, closs: 9998.12, aloss: 0.17, return: 100.0
Run: 352,

Run: 354, Day: 12, closs: 9998.01, aloss: 0.16, return: 100.0
Run: 354, Day: 13, closs: 9998.01, aloss: 0.16, return: 100.0
Run: 354, Day: 14, closs: 9998.01, aloss: 0.16, return: 100.0
Run: 354, Day: 15, closs: 9998.01, aloss: 0.16, return: 100.0
Run: 354, Day: 16, closs: 9998.01, aloss: 0.16, return: 100.0
Run: 354, Day: 17, closs: 9998.01, aloss: 0.16, return: 100.0
Run: 354, Day: 18, closs: 9998.01, aloss: 0.16, return: 100.0
Run: 354, Day: 19, closs: 9998.01, aloss: 0.16, return: 100.0
Run: 354, Day: 20, closs: 9998.01, aloss: 0.16, return: 100.0
Run: 354, Day: 21, closs: 9998.01, aloss: 0.16, return: 100.0
Run: 354, Day: 22, closs: 9998.03, aloss: 0.16, return: 100.0
Run: 354, Day: 23, closs: 9998.02, aloss: 0.16, return: 100.0
Run: 354, Day: 24, closs: 9998.03, aloss: 0.16, return: 100.0
Run: 354, Day: 25, closs: 9998.03, aloss: 0.16, return: 100.0
Run: 354, Day: 26, closs: 9998.03, aloss: 0.16, return: 100.0
Run: 354, Day: 27, closs: 9998.03, aloss: 0.16, return: 100.0
Run: 354

KeyboardInterrupt: 

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
huber_loss = keras.losses.Huber()
action_values_history = []
critic_value_history = []
rewards_history = []
running_reward = 0
episode_count = 0
best_reward = 0

while True:  # Run until solved
    state = env.reset()
    episode_reward = 0
    with tf.GradientTape() as tape:
        for timestep in range(1, max_steps_per_episode):
            # Adding this line would show the attempts
            # of the agent in a pop up window.

            state = tf.convert_to_tensor(state)
            state = tf.expand_dims(state, 0)

            # Predict action probabilities and estimated future rewards
            # from environment state
            action_value, critic_value = model(state)
            critic_value_history.append(critic_value[0, 0])

            action_value = tf.squeeze(action_value, axis=0)

            action_values_history.append(action_value)

            # Apply the sampled action in our environment
            state, reward, done, _ = env.step(action_value)
            rewards_history.append(reward)

            if done:
                break

        # Calculate expected value from rewards
        # - At each timestep what was the total reward received after that timestep
        # - Rewards in the past are discounted by multiplying them with gamma
        # - These are the labels for our critic
        returns = []
        discounted_sum = 0
        for r in rewards_history[::-1]:
            discounted_sum = r + gamma * discounted_sum
            returns.insert(0, discounted_sum)

        # Normalize
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
        returns = returns.tolist()

        # Calculating loss values to update our network
        history = zip(action_values_history, critic_value_history, returns)
        actor_losses = []
        critic_losses = []
        for action_values, critic_value, ret in history:
            print(critic_value)
            # At this point in history, the critic estimated that we would get a
            # total reward = `value` in the future. We took an action with log probability
            # of `log_prob` and ended up recieving a total reward = `ret`.
            # The actor must be updated so that it predicts an action that leads to
            # high rewards (compared to critic's estimate) with high probability.
            diff = ret - critic_value
            actor_losses.append(-action_value * diff)  # actor loss

            # The critic must be updated so that it predicts a better estimate of
            # the future rewards.
            critic_losses.append(
                huber_loss(tf.expand_dims(value, 0), tf.expand_dims(ret, 0))
            )
            
        # Backpropagation
        loss_value = sum(actor_losses) + sum(critic_losses)
#         print(f'Actor loss: {sum(actor_losses)}')
        print(f'Critic loss: {sum(critic_losses)}')
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Clear the loss and reward history
        action_values_history.clear()
        critic_value_history.clear()
        rewards_history.clear()

    # Log details
    episode_count += 1
    template = "reward: {:.2f} at episode {}"
    print(template.format(reward, episode_count))
    
    if reward > best_reward:
        model.save('models/agent.h5')
#         env.render()
        best_reward = reward

In [11]:
agent_model = keras.models.load_model(f'models/agent.h5')

symbols = [symbol for symbol in list(data_df) if '_grad' not in symbol]


state = env.reset()
for timestep in range(1, max_steps_per_episode):
    state = tf.convert_to_tensor(state)
    state = tf.expand_dims(state, 0)
    
    action_value, critic_value = model(state)

    action_value = tf.squeeze(action_value, axis=0)

    state, reward, done, _ = env.step(action_value)

    print()
    print('STATE:')
    for idx, symbol in enumerate(symbols):
        if action_value[idx] < (1 / 3):
            decision = 'SELL'
        elif action_value[idx] < (2 / 3):
            decision = 'HOLD'
        else:
            decision = 'BUY'

        print(f"{symbol} price: {round(state[idx], 5)}$")
        print(f"{symbol} grad: {round(state[idx + len(symbols)], 5)}")
        print(f"{symbol} held: {round(state[idx + len(symbols) * 2], 5)}$")
        print(f"{symbol} decision: {decision}")
        print(f"{symbol} amount: {action_value[idx + len(symbols)]}")
        print()
    print('NEW STATE')
    env.render()
    input()



STATE:
AAPL price: 0.50081$
AAPL grad: 0.0


IndexError: index 54 is out of bounds for axis 0 with size 54